In [28]:
import numpy as np
import csv
import timeit
from scipy.stats import norm
import pandas as pd

In [29]:
# 初始化数据
data = pd.read_csv('height_data.csv',header=0)
data = data.values
data.shape

(2000, 1)

In [30]:
# 初始化模型参数
n = data.shape[0]
pi1 = 0.5
pi2 = 0.5
mu1 = np.random.randn()
mu2 = np.random.randn()
sigma1 = np.random.rand()
sigma2 = np.random.rand()
print(mu1,mu2,sigma1,sigma2)

0.2890518982148502 1.656051772353249 0.4646197104428297 0.14269063901211199


In [31]:
# 定义高斯分布的概率密度函数
def gaussian(x, mu, sigma):
    return norm.logpdf(x, mu, sigma)

In [32]:
# 定义E步骤
def e_step(data, pi1, pi2, mu1, mu2, sigma1, sigma2):
    gamma1 = pi1 * gaussian(data, mu1, sigma1)
    gamma2 = pi2 * gaussian(data, mu2, sigma2)
    gamma_sum = gamma1 + gamma2
    gamma1 = gamma1 / gamma_sum
    gamma2 = gamma2 / gamma_sum
    return gamma1, gamma2

In [33]:
# 定义M步骤
def m_step(data, gamma1, gamma2):
    n1 = np.sum(gamma1)
    n2 = np.sum(gamma2)
    pi1 = n1 / data.shape[0]
    pi2 = n2 / data.shape[0]
    mu1 = np.sum(gamma1 * data) / n1
    mu2 = np.sum(gamma2 * data) / n2
    sigma1 = np.sqrt(np.sum(gamma1 * (data - mu1) ** 2) / n1)
    sigma2 = np.sqrt(np.sum(gamma2 * (data - mu2) ** 2) / n2)
    return pi1, pi2, mu1, mu2, sigma1, sigma2

In [34]:
# 定义EM算法
def em_algorithm(data, pi1, pi2, mu1, mu2, sigma1, sigma2, max_iter=100, tol=1e-4):
    for i in range(max_iter):
        gamma1, gamma2 = e_step(data, pi1, pi2, mu1, mu2, sigma1, sigma2)
        pi1_new, pi2_new, mu1_new, mu2_new, sigma1_new, sigma2_new = m_step(data, gamma1, gamma2)
        if np.abs(pi1_new - pi1) < tol and np.abs(pi2_new - pi2) < tol and np.abs(mu1_new - mu1) < tol \
                and np.abs(mu2_new - mu2) < tol and np.abs(sigma1_new - sigma1) < tol and np.abs(sigma2_new - sigma2) < tol:
            break
        pi1, pi2, mu1, mu2, sigma1, sigma2 = pi1_new, pi2_new, mu1_new, mu2_new, sigma1_new, sigma2_new
    return pi1, pi2, mu1, mu2, sigma1, sigma2

In [35]:
# 运行EM算法
pi1, pi2, mu1, mu2, sigma1, sigma2 = em_algorithm(data, pi1, pi2, mu1, mu2, sigma1, sigma2)
print(type(pi1))

<class 'numpy.float64'>


In [36]:
# 预测数据
gamma1, gamma2 = e_step(data, pi1, pi2, mu1, mu2, sigma1, sigma2)
y_pred = np.argmax(np.array([gamma1 - gamma1, gamma2]), axis=0) + 1

#计算均方误差和对数似然
mse = np.mean((y_pred - data) ** 2)
log_likelihood = np.sum(np.log(pi1 * gaussian(data, mu1, sigma1) + pi2 * gaussian(data, mu2, sigma2)))

#输出模型参数和指标
print("pi1: ", pi1)
print("pi2: ", pi2)
print("mu1: ", mu1)
print("mu2: ", mu2)
print("sigma1: ", sigma1)
print("sigma2: ", sigma2)
print("MSE: ", mse)
print("Log-likelihood: ", log_likelihood)

pi1:  0.03095518607068831
pi2:  0.9690448139293116
mu1:  173.06964342498824
mu2:  173.0696660732229
sigma1:  6.962874756994861
sigma2:  6.962870329767918
MSE:  29313.311975674475
Log-likelihood:  nan


C:\Users\hangxiaofan1873\AppData\Local\Temp\ipykernel_21304\1077499231.py:7: RuntimeWarning: invalid value encountered in log
  log_likelihood = np.sum(np.log(pi1 * gaussian(data, mu1, sigma1) + pi2 * gaussian(data, mu2, sigma2)))


In [27]:
log_pdf_value = norm.logpdf(170, -2.4, 0.36)
log_pdf_value

-114667.18123790296